<a href="https://colab.research.google.com/github/bcury/Machine-Learning-Projects/blob/main/PySpark_VectorAssembler_PCA_Binarizer_OneHotEncoding_ImputerMissingValues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 40 kB/s 
     |████████████████████████████████| 198 kB 77.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=7548da81adfd5dc1444a88bf767d539111893f61266846ad858631bd8311a5e9
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('Importação').getOrCreate()

In [4]:
arqschema = 'id INT, nome STRING, status STRING, cidade STRING, vendas INT, data DATE'

In [5]:
despachantes = spark.read.csv("despachantes.csv", header=False, schema=arqschema)
despachantes.show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [6]:
despachantes.schema

StructType(List(StructField(id,IntegerType,true),StructField(nome,StringType,true),StructField(status,StringType,true),StructField(cidade,StringType,true),StructField(vendas,IntegerType,true),StructField(data,DateType,true)))

In [7]:
# mesmo arquivo mas em parquet

In [8]:
par = spark.read.format('parquet').load('despachantes.parquet')
par.show(5)

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
+---+-------------------+-----+-------------+---+----------+
only showing top 5 rows



In [9]:
# json

In [10]:
json = spark.read.format('json').load('despachantes.json')
json.show(5)

+-------------+-----------+---+-------------------+------+------+
|       cidade|       data| id|               nome|status|vendas|
+-------------+-----------+---+-------------------+------+------+
|  Santa Maria| 2020-08-11|  1|   Carminda Pestana| Ativo|    23|
|Novo Hamburgo| 2020-03-05|  2|    Deolinda Vilela| Ativo|    34|
| Porto Alegre| 2020-02-05|  3|   Emídio Dornelles| Ativo|    34|
| Porto Alegre| 2020-02-05|  4|Felisbela Dornelles| Ativo|    36|
| Porto Alegre| 2020-02-05|  5|     Graça Ornellas| Ativo|    12|
+-------------+-----------+---+-------------------+------+------+
only showing top 5 rows



In [11]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('VectorAssembler').getOrCreate()

In [7]:
from pyspark.ml.feature import VectorAssembler

In [14]:
carros = spark.read.csv('Carros.csv', header=True, inferSchema=True, sep=';')
carros.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

vamos criar a coluna caracteristicas: que  tem todos os inputs

In [18]:
vectas = VectorAssembler(inputCols=['Consumo','Cilindros','Cilindradas','RelEixoTraseiro','Peso','Tempo','TipoMotor','Transmissao','Marchas','Carburadors'], outputCol='caracteristicas')
carros = vectas.transform(carros)
carros.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+--------------------+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|     caracteristicas|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+--------------------+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|[21.0,6.0,160.0,3...|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|[21.0,6.0,160.0,3...|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|[228.0,4.0,108.0,...|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|[214.0,6.0,258.0,...|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|[187.0,8.0,360

In [22]:
carros.select('caracteristicas').show(truncate=False) #pra nao truncar

+-----------------------------------------------------+
|caracteristicas                                      |
+-----------------------------------------------------+
|[21.0,6.0,160.0,39.0,262.0,1646.0,0.0,1.0,4.0,4.0]   |
|[21.0,6.0,160.0,39.0,2875.0,1702.0,0.0,1.0,4.0,4.0]  |
|[228.0,4.0,108.0,385.0,232.0,1861.0,1.0,1.0,4.0,1.0] |
|[214.0,6.0,258.0,308.0,3215.0,1944.0,1.0,0.0,3.0,1.0]|
|[187.0,8.0,360.0,315.0,344.0,1702.0,0.0,0.0,3.0,2.0] |
|[181.0,6.0,225.0,276.0,346.0,2022.0,1.0,0.0,3.0,1.0] |
|[143.0,8.0,360.0,321.0,357.0,1584.0,0.0,0.0,3.0,4.0] |
|[244.0,4.0,1467.0,369.0,319.0,20.0,1.0,0.0,4.0,2.0]  |
|[228.0,4.0,1408.0,392.0,315.0,229.0,1.0,0.0,4.0,2.0] |
|[192.0,6.0,1676.0,392.0,344.0,183.0,1.0,0.0,4.0,4.0] |
|[178.0,6.0,1676.0,392.0,344.0,189.0,1.0,0.0,4.0,4.0] |
|[164.0,8.0,2758.0,307.0,407.0,174.0,0.0,0.0,3.0,3.0] |
|[173.0,8.0,2758.0,307.0,373.0,176.0,0.0,0.0,3.0,3.0] |
|[152.0,8.0,2758.0,307.0,378.0,18.0,0.0,0.0,3.0,3.0]  |
|[104.0,8.0,472.0,293.0,525.0,1798.0,0.0,0.0,3.0

PCA

In [3]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('PCA').getOrCreate()

In [5]:
from pyspark.ml.feature  import PCA

In [6]:
carros = spark.read.csv('Carros.csv', header=True, inferSchema=True, sep=';')
carros.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [8]:
vectas = VectorAssembler(inputCols=['Consumo','Cilindros','Cilindradas','RelEixoTraseiro','Peso','Tempo','TipoMotor','Transmissao','Marchas','Carburadors'], outputCol='caracteristicas')
carros = vectas.transform(carros)
carros.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+--------------------+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|     caracteristicas|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+--------------------+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|[21.0,6.0,160.0,3...|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|[21.0,6.0,160.0,3...|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|[228.0,4.0,108.0,...|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|[214.0,6.0,258.0,...|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|[187.0,8.0,360

In [9]:
carros.select('caracteristicas').show(truncate=False) #pra nao truncar

+-----------------------------------------------------+
|caracteristicas                                      |
+-----------------------------------------------------+
|[21.0,6.0,160.0,39.0,262.0,1646.0,0.0,1.0,4.0,4.0]   |
|[21.0,6.0,160.0,39.0,2875.0,1702.0,0.0,1.0,4.0,4.0]  |
|[228.0,4.0,108.0,385.0,232.0,1861.0,1.0,1.0,4.0,1.0] |
|[214.0,6.0,258.0,308.0,3215.0,1944.0,1.0,0.0,3.0,1.0]|
|[187.0,8.0,360.0,315.0,344.0,1702.0,0.0,0.0,3.0,2.0] |
|[181.0,6.0,225.0,276.0,346.0,2022.0,1.0,0.0,3.0,1.0] |
|[143.0,8.0,360.0,321.0,357.0,1584.0,0.0,0.0,3.0,4.0] |
|[244.0,4.0,1467.0,369.0,319.0,20.0,1.0,0.0,4.0,2.0]  |
|[228.0,4.0,1408.0,392.0,315.0,229.0,1.0,0.0,4.0,2.0] |
|[192.0,6.0,1676.0,392.0,344.0,183.0,1.0,0.0,4.0,4.0] |
|[178.0,6.0,1676.0,392.0,344.0,189.0,1.0,0.0,4.0,4.0] |
|[164.0,8.0,2758.0,307.0,407.0,174.0,0.0,0.0,3.0,3.0] |
|[173.0,8.0,2758.0,307.0,373.0,176.0,0.0,0.0,3.0,3.0] |
|[152.0,8.0,2758.0,307.0,378.0,18.0,0.0,0.0,3.0,3.0]  |
|[104.0,8.0,472.0,293.0,525.0,1798.0,0.0,0.0,3.0

In [10]:
pca = PCA(k=3, inputCol='caracteristicas', outputCol='caracteristicas_pca')
modelo = pca.fit(carros) # k = 3 caract artificiais

In [13]:
resultado = modelo.transform(carros)
resultado.select('caracteristicas_pca').show(truncate=False)

+-----------------------------------------------------------+
|caracteristicas_pca                                        |
+-----------------------------------------------------------+
|[618.7707206779614,-937.712394997354,1231.9633529945509]   |
|[3112.9887675342206,-161.05746385491523,1191.861991305438] |
|[640.4959007710695,-1120.7188865110418,1320.0756315189049] |
|[3466.0956877556678,-149.69421418298342,1401.2041780368531]|
|[661.4577445758732,-812.4592128844115,1395.2949328316356]  |
|[769.234367178774,-1120.4160559477316,1518.7436632279525]  |
|[644.8369503533214,-727.9539376169615,1313.681521097935]   |
|[9.101880661709801,1061.295403667789,1045.1710500215693]   |
|[67.13360966508397,878.479368204501,1143.9379120496164]    |
|[31.390504477140617,1095.3694498285743,1306.0124861906327] |
|[32.89165922208959,1091.1521230845228,1310.0881577350906]  |
|[-118.37273751675397,1832.771927405815,2088.6955393326043] |
|[-150.18148405358022,1820.8730926512776,2091.1033550766124]|
|[-184.0

**Binarizer**

In [14]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('Binarizer').getOrCreate()

In [16]:
from pyspark.ml.feature import Binarizer

In [20]:
iris = spark.read.csv('iris.csv', header=True, inferSchema=True)
iris.show()

+-----------+----------+-----------+----------+-----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|
+-----------+----------+-----------+----------+-----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|
|        5.4|       3.7|        1.5|       0.2|Iris-setosa|
|        4.8|       3.4|        1.6|       0.2|Iris-setosa|
|        4.8|       3.0|        1.4|       0.1|Iris-setosa|
|        4.3|       3.0|        1.1|    

In [21]:
binarizer = Binarizer(threshold=5, inputCol='sepallength', outputCol='sepallenght_bin')

In [23]:
iris_bin = binarizer.transform(iris)
iris_bin.show() # se o valor de sepallenght for maior que 5, sepallength_bin recebe 1, se menor, 0

+-----------+----------+-----------+----------+-----------+---------------+
|sepallength|sepalwidth|petallength|petalwidth|      class|sepallenght_bin|
+-----------+----------+-----------+----------+-----------+---------------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|            1.0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|            0.0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|            0.0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|            0.0|
|        5.0|       3.6|        1.4|       0.2|Iris-setosa|            0.0|
|        5.4|       3.9|        1.7|       0.4|Iris-setosa|            1.0|
|        4.6|       3.4|        1.4|       0.3|Iris-setosa|            0.0|
|        5.0|       3.4|        1.5|       0.2|Iris-setosa|            0.0|
|        4.4|       2.9|        1.4|       0.2|Iris-setosa|            0.0|
|        4.9|       3.1|        1.5|       0.1|Iris-setosa|            0.0|
|        5.4

**OneHotEncoding**

Pra ele funcionar, a coluna tem que estar númerica, podemos transformar strings em numeros com o LabelEncoding ou StringIndexer

In [24]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('onehotencoding').getOrCreate()

In [25]:
from pyspark.ml.feature import OneHotEncoder

In [29]:
churn = spark.read.csv('Churn.csv',sep=';',inferSchema=True, header=True)
churn.show()

+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        619|   France|Female| 42|     2|       0|            1|        1|             1|       10134888|     1|
|        608|    Spain|Female| 41|     1| 8380786|            1|        0|             1|       11254258|     0|
|        502|   France|Female| 42|     8| 1596608|            3|        1|             0|       11393157|     1|
|        699|   France|Female| 39|     1|       0|            2|        0|             0|        9382663|     0|
|        850|    Spain|Female| 43|     2|12551082|            1|        1|             1|         790841|     0|
|        645|    Spain|  Male| 44|     8|11375578|            2|        1|             0|       

In [30]:
from pyspark.ml.feature import StringIndexer

In [32]:
indice = StringIndexer(inputCol='Geography', outputCol='indice')
modelo = indice.fit(churn)
dados_indices = modelo.transform(churn)
dados_indices.select('Geography','indice').show()

+---------+------+
|Geography|indice|
+---------+------+
|   France|   0.0|
|    Spain|   2.0|
|   France|   0.0|
|   France|   0.0|
|    Spain|   2.0|
|    Spain|   2.0|
|   France|   0.0|
|  Germany|   1.0|
|   France|   0.0|
|   France|   0.0|
|   France|   0.0|
|    Spain|   2.0|
|   France|   0.0|
|   France|   0.0|
|    Spain|   2.0|
|  Germany|   1.0|
|  Germany|   1.0|
|    Spain|   2.0|
|    Spain|   2.0|
|   France|   0.0|
+---------+------+
only showing top 20 rows



In [33]:
indice = StringIndexer(inputCol='Geography', outputCol='indexer_c1')
indiceonehot = indice.fit(churn).transform(churn)

indice = StringIndexer(inputCol='Gender', outputCol='indexer_c2')
indiceonehot = indice.fit(indiceonehot).transform(indiceonehot)


In [34]:
indiceonehot.select('indexer_c1','indexer_c2').show()

+----------+----------+
|indexer_c1|indexer_c2|
+----------+----------+
|       0.0|       1.0|
|       2.0|       1.0|
|       0.0|       1.0|
|       0.0|       1.0|
|       2.0|       1.0|
|       2.0|       0.0|
|       0.0|       0.0|
|       1.0|       1.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       0.0|       0.0|
|       2.0|       0.0|
|       0.0|       1.0|
|       0.0|       1.0|
|       2.0|       1.0|
|       1.0|       0.0|
|       1.0|       0.0|
|       2.0|       1.0|
|       2.0|       0.0|
|       0.0|       1.0|
+----------+----------+
only showing top 20 rows



In [36]:
onehot = OneHotEncoder(inputCols=['indexer_c1','indexer_c2'],outputCols=['onehot_c1','onehot_c2'])

In [39]:
modelo = onehot.fit(indiceonehot)
onehot_out = modelo.transform(indiceonehot)
onehot_out.select('onehot_c1','onehot_c2').show(truncate=False)

+-------------+-------------+
|onehot_c1    |onehot_c2    |
+-------------+-------------+
|(2,[0],[1.0])|(1,[],[])    |
|(2,[],[])    |(1,[],[])    |
|(2,[0],[1.0])|(1,[],[])    |
|(2,[0],[1.0])|(1,[],[])    |
|(2,[],[])    |(1,[],[])    |
|(2,[],[])    |(1,[0],[1.0])|
|(2,[0],[1.0])|(1,[0],[1.0])|
|(2,[1],[1.0])|(1,[],[])    |
|(2,[0],[1.0])|(1,[0],[1.0])|
|(2,[0],[1.0])|(1,[0],[1.0])|
|(2,[0],[1.0])|(1,[0],[1.0])|
|(2,[],[])    |(1,[0],[1.0])|
|(2,[0],[1.0])|(1,[],[])    |
|(2,[0],[1.0])|(1,[],[])    |
|(2,[],[])    |(1,[],[])    |
|(2,[1],[1.0])|(1,[0],[1.0])|
|(2,[1],[1.0])|(1,[0],[1.0])|
|(2,[],[])    |(1,[],[])    |
|(2,[],[])    |(1,[0],[1.0])|
|(2,[0],[1.0])|(1,[],[])    |
+-------------+-------------+
only showing top 20 rows



Imputer(missing values)

In [40]:
import pyspark, findspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName('missing_values').getOrCreate()

In [41]:
from pyspark.ml.feature import Imputer

In [47]:
carro_nan = spark.read.csv('CarrosNAN.csv', header=True, sep=';', inferSchema=True)
carro_nan.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|       null|             39|2875| null|        0|          1|      4|          4|110|
|    228|        0|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        0|       null|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        0|        360|            315|null| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| null|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [55]:
imput = Imputer(inputCols=['Cilindradas','Peso'], outputCols=['nova_cilindrada','novo_peso'])

In [58]:
modelo = imput.fit(carro_nan)
carros_sem_nan = modelo.transform(carro_nan)
carros_sem_nan.select('nova_cilindrada','novo_peso').show() #preencheu NAN com valor da média que é padrão

+---------------+---------+
|nova_cilindrada|novo_peso|
+---------------+---------+
|            160|      262|
|            848|     2875|
|            108|      232|
|            848|     3215|
|            360|     1318|
|            225|      346|
|            360|      357|
|           1467|      319|
|           1408|      315|
|           1676|      344|
|           1676|     1318|
|           2758|      407|
|           2758|      373|
|           2758|      378|
|            472|      525|
|            848|     5424|
|            440|     5345|
|            787|       22|
|            757|     1615|
|            711|     1835|
+---------------+---------+
only showing top 20 rows



Para substituir pela mediana por exemplo, basta usar:
imput =  Imputer(inputcols=[], outputcols =[])
imput.setStrategy('median').setMissingValue(0).fit(carro_nan).transform(carro_nan).select('coluna que quer substituir)